# Content
## 0. Importul bibliotecilor
## 1. Importul datelor
## 2. Completarea sau excluderea datelor lipsa (data imputation)
## 3. Divizarea datelor in date feature si date labels (X si y)
## 4. Conversia valorilor non-numetice in numerice (feature encoding)
## 5. Implementarea modelului si realizarea predictiei


# 0. Importul bibliotecilor

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 1. Importul datelor

In [2]:
# importul datelor
cars=pd.read_csv('car-sales-data.csv')

In [3]:
cars.head(10)

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
5,Honda,Red,42652.0,4.0,23883.0
6,Toyota,Blue,163453.0,4.0,8473.0
7,Honda,White,NaN,4.0,20306.0
8,NaN,White,130538.0,4.0,9374.0
9,Honda,Blue,51029.0,4.0,26683.0


In [4]:
# Verificare dimensiunii datelor
cars.shape

(1000, 5)

# 2. Completarea sau excluderea datelor lipsa

In [5]:
# Verificare datelor lipsa in fiecare coloana
cars.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [6]:
# Verificare tipurilor de date in fiecare coloana
cars.dtypes

Make              object
Colour            object
Odometer (KM)    float64
Doors            float64
Price            float64
dtype: object

## 2.1 Completarea datelor lipsa utilizand Pandas

In [7]:
# Completarea valorilor lipsa in coloana "Make" cu valoarea "missing"
cars['Make'].fillna('missing', inplace=True)

In [8]:
# Completarea valorilor lipsa in coloana "Colour" cu valoarea "missing"
cars['Colour'].fillna('missing', inplace=True)

In [9]:
# Completarea valorilor lipsa in coloana "Odometer (KM)" cu valoarea medie
cars['Odometer (KM)'].fillna(cars["Odometer (KM)"].mean(), inplace=True)

In [10]:
# verificarea valorii median a coloanei "Doors"
cars['Doors'].median()

4.0

In [11]:
# Completarea valorilor lipsa in coloana "Doors" cu valoarea median (4)
cars['Doors'].fillna(4, inplace=True)

In [12]:
# Verificare datelor lipsa 
cars.isna().sum()

Make              0
Colour            0
Odometer (KM)     0
Doors             0
Price            50
dtype: int64

In [13]:
# Excluderea datelor care in coloana Price (coloana label) au lipsa
cars.dropna(inplace=True)

In [14]:
# Verificare datelor lipsa 
cars.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
Price            0
dtype: int64

In [15]:
# Verificare dimensiunii datelor
cars.shape

(950, 5)

## 2.2 Completarea datelor lipsa utilizand Scikit-Learn (imputation)

*necesita divizarea datelor

In [16]:
# Reimportul datelor pentru a avea date lipsa
cars=pd.read_csv('car-sales-data.csv')

In [17]:
# Verificare datelor lipsa in fiecare coloana
cars.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [18]:
# Excluderea datelor care in coloana Price (coloana label) au lipsa
cars.dropna(subset=['Price'], inplace=True)

In [19]:
# Verificare datelor lipsa in fiecare coloana
cars.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [20]:
# Verificare dimensiunii datelor
cars.shape

(950, 5)

In continuare completarea datelor lipsa in celelalte coloane se va realiza pe fiecare set (training sau test) in parte de a ceea se va trece la subpunctul c) ca mai apoi se va reveni la b2)

# 3. Divizarea datelor in date feature si date labels (X si y)

In [21]:
# Divizarea datelor (coloanelor) in date feature X (caracteristici) si
# date labels y (rezultat)
np.random.seed(9)
X = cars.drop('Price', axis=1)
y = cars['Price']

In [22]:
# Divizarea datelor in setul de training si setul de test
np.random.seed(9)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [23]:
# Vizualizarea dimensiunilor seturilor
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((760, 4), (190, 4), (760,), (190,))

## 2.2 Completarea datelor lipsa utilizand Scikit-Learn (continuare)

In [24]:
# Importul functiilor de completare (imputare) si de transformare
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [25]:
# Definirea coloanelor "Make" si "Colour" de tip categorii
cat_features=["Make", "Colour"]

In [26]:
# Definirea coloanei "Doors" de tip categorie cu valori numerice
door_feature = ["Doors"]

In [27]:
# Definirea coloanei "Odometer (KM)" de tip  numeric
num_features = ["Odometer (KM)"]

In [28]:
# Elaborarea modelului de completare a datelor lipsa a coloanelor de tip categorii 
cat_imputer=SimpleImputer(strategy="constant", fill_value="missing")

In [29]:
# Elaborarea modelului de completare a datelor lipsa a coloanei de tip categorie cu valori numerice 
door_imputer = SimpleImputer(strategy="median")

In [30]:
# Elaborarea modelului de completare a datelor lipsa a coloanei de tip numeric   
num_imputer = SimpleImputer(strategy="mean")

In [31]:
# Crearea unui imputer care v-a realiza completarea datelor lipsa
imputer = ColumnTransformer([
    ("cat_imputer", cat_imputer, cat_features),
    ("door_imputer", door_imputer, door_feature),
    ("num_imputer", num_imputer, num_features)
])

In [32]:
# Completarea datelor lipsa in setul de training
X_train = imputer.fit_transform(X_train)

In [33]:
# Completarea datelor lipsa in setul de test
X_test = imputer.fit_transform(X_test)

In [34]:
# Vizualizarea datelor de training
X_train = pd.DataFrame(X_train, columns = ['Make', 'Colour', 'Doors', 'Odometer (KM)'] )
X_train.head()

,Make,Colour,Doors,Odometer (KM)
0,Toyota,Black,4.0,90508.0
1,Nissan,Black,4.0,240536.0
2,Toyota,Black,4.0,197498.0
3,Honda,missing,4.0,178774.0
4,BMW,Red,5.0,131587.0


In [35]:
# Vizualizarea datelor de test
X_test = pd.DataFrame(X_test, columns = ['Make', 'Colour', 'Doors', 'Odometer (KM)'])
X_test.head()

,Make,Colour,Doors,Odometer (KM)
0,Honda,White,4.0,195787.0
1,missing,White,3.0,152006.0
2,Nissan,Blue,3.0,234146.0
3,Honda,missing,4.0,136926.361111
4,Toyota,Green,4.0,172493.0


In [36]:
# Verificare datelor lipsa in fiecare coloana a setului de training
X_train.isna().sum()

Make             0
Colour           0
Doors            0
Odometer (KM)    0
dtype: int64

In [37]:
# Verificare datelor lipsa in fiecare coloana a setului de test
X_test.isna().sum()

Make             0
Colour           0
Doors            0
Odometer (KM)    0
dtype: int64

In [38]:
# Vizualizarea dimensiunilor seturilor
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((760, 4), (190, 4), (760,), (190,))

# 4.  Conversia valorilor non-numerice in numerice (feature encoding)

In [39]:
# Verificare tipurilor de date in fiecare coloana a setului de training
X_train.dtypes

Make             object
Colour           object
Doors            object
Odometer (KM)    object
dtype: object

In [40]:
# Verificare tipurilor de date in fiecare coloana a setului de test
X_test.dtypes

Make             object
Colour           object
Doors            object
Odometer (KM)    object
dtype: object

In [41]:
# Verificare tipurilor de date in setul label de training
y_train.dtypes

dtype('float64')

In [42]:
# Verificare tipurilor de date in setul label de test
y_test.dtypes

dtype('float64')

In [43]:
# Conversia tipurilor de date din coloanele numerice in tip corespunzator in setul de training
X_train["Odometer (KM)"] = X_train["Odometer (KM)"].astype(float)
X_train["Doors"] = X_train["Doors"].astype(int)

In [44]:
# Verificare tipurilor de date in fiecare coloana a setului de training
X_train.dtypes

Make              object
Colour            object
Doors              int32
Odometer (KM)    float64
dtype: object

In [45]:
# Conversia tipurilor de date din coloanele numerice in tip corespunzator in setul de test
X_test["Odometer (KM)"] = X_test["Odometer (KM)"].astype(float)
X_test["Doors"] = X_test["Doors"].astype(int)

In [46]:
# Verificare tipurilor de date in fiecare coloana a setului de test
X_test.dtypes

Make              object
Colour            object
Doors              int32
Odometer (KM)    float64
dtype: object

In [47]:
# importul functiilor necesare pentru conversia datelor de tip categorie in tip numeric
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [48]:
# definirea coloanelor de tipul categorie (inclusiv cele de tip categorie cu valori numerice)
categorical_features = ["Make", "Colour", "Doors"]

In [49]:
# instantierea unui codor ce ar realiza conversia datelor de tip categorie in date de tip numeric
one_hot = OneHotEncoder()

In [50]:
# aplicarea codorului asupra coloanelor de tip categorie si neglijarea celorlalte
transformer = ColumnTransformer([("one_hot", 
                                  one_hot, 
                                  categorical_features)],
                                  remainder="passthrough")

In [51]:
# implementarea conversiei in setul de training
X_train = transformer.fit_transform(X_train)
X_train

<760x15 sparse matrix of type '<class 'numpy.float64'>'
	with 3040 stored elements in Compressed Sparse Row format>

In [52]:
# implementarea conversiei in setul de test
X_test = transformer.fit_transform(X_test)
X_test

<190x15 sparse matrix of type '<class 'numpy.float64'>'
	with 760 stored elements in Compressed Sparse Row format>

# 5. Implementarea modelului si realizarea predictiei

In [53]:
from sklearn.ensemble import RandomForestRegressor
np.random.seed(9)
model = RandomForestRegressor()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.35336079981377755